# Лабораторная работа 8. Анализ одноканальной системы массового обслуживания с учетом приоритета заявок
Выполнил студент РТ5-81Б Пакало Александр Сергеевич

Задание 1.
Система электронного документооборота с автоматическим классификатором
поступающей корреспонденции должна учитывать различную важность документов,
количество типов документов $P$,
средняя интенсивность поступления заявок
$\lambda_1, \lambda_2, \dots, \lambda_P$,
средняя интенсивность обслуживания
$\mu_1, \mu_2, \dots, \mu_P$ соответственно, очередь неограничена.
Приоритеты заявок
$p_1 < p_2 < \dots < p_P$.

Оценить среднее время нахождения каждого типа заявки в очереди (теоретически
и экспериментально) для одного из 4 случаев:

- относительный фиксированный приоритет (V1);
- абсолютный фиксированный приоритет (V2);
- приоритет с зависимой задержкой без прерывания (V3);
- приоритет с зависимой задержкой с прерываниями (V4);

в соответствии с вариантом.

In [1]:
Variant <- 5
set.seed(Variant)
V <- sample(c("V1", "V2", "V3", "V4"), 1)
P <- sample(c(4, 6), 1)
if ((V == "V3") | (V == "V4")) {
    b <- sort(sample(c(1:10), P))
}
lambda <- runif(P)
mu <- runif(P, 1, 3)
View(data.frame(P, V))
if ((V == "V3") | (V == "V4")) {
    View(data.frame(lambda, mu, b))
}
if ((V == "V1") | (V == "V2")) {
    View(data.frame(lambda, mu))
}

P,V
<dbl>,<chr>
4,V2


lambda,mu
<dbl>,<dbl>
0.9168758,2.055920
0.2843995,2.615870
0.1046501,2.913000
0.7010575,1.220906


In [3]:
sum(lambda / mu)

[1] 0.6970042

Как видно, суммарная нагрузка на систему превышает 1, следовательно будет
накапливаться бесконечная очередь. Немного изменим
исходные данные.

In [4]:
lambda[1] <- 0.1
lambda[2] <- 0.1
ro <- sum(lambda / mu)
ro

[1] 0.6970042

### Теоретически
$$\lambda=\sum_{p=1}^P\lambda_p$$

In [5]:
lambda_ <- sum(lambda)
lambda_

[1] 1.005708

$$\frac{1}{\mu}=\sum_{p=1}^P \frac{\lambda_p}{\lambda}\cdot \frac{1}{\mu_p}$$

In [7]:
ro <- lambda / mu

mu_ <- sum(ro) / lambda_
mu_

[1] 0.6930486

$$\rho=\frac{\lambda}{\mu}$$

In [8]:
ro_ <- lambda_ / mu_
ro_

[1] 1.451136

$$W_0=\sum_{p=1}^P\frac{\rho_p}{\mu_p}$$

In [9]:
W0 <- sum(ro / mu)
W0

[1] 0.5209205

Для получения $W_p$ воспользуемся формулой

$$W_p= \frac{\frac{\rho_p}{\mu_p}+\sum_{i=p+1}^P \rho_i\cdot \left(\frac{1}{\mu_p}+\frac{1}{\mu_i}\right)+\sum_{i=p+1}^P\rho_i\cdot W_i}{1-\sum_{i=p}^P \rho_i}$$

In [10]:
get_Wqueue <- function(p) {
    numerator <- ro[p] / mu[p]
    denominator <- 1 - sum(ro[p:P])

    if (p == P) {
        return(numerator / denominator)
    }

    sum_part1 <- sum(unlist(
        lapply(
            c((p + 1):P),
            function(i) ro[i] * (1 / mu[p] + 1 / mu[i])
        )
    ))

    sum_part2 <- sum(unlist(
        lapply(
            c((p + 1):P),
            function(i) ro[i] * get_Wqueue(i)
        )
    ))

    result <- (numerator + sum_part1 + sum_part2) / denominator


    if (result < 0) {
        return(Inf)
    }

    return(result)
}

results <- data.frame(theoretical = unlist(lapply(1:P, get_Wqueue)))
row.names(results) <- 1:P
results

,theoretical
,<dbl>
1,5.786096
2,4.225534
3,3.370473
4,1.104573


### Численно

In [11]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

Loading required package: simmer



simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [12]:
create_trajectory <- function(mu, name) {
    return(trajectory(name) %>%
        seize("server", 1) %>%
        timeout(function() rexp(1, mu)) %>%
        release("server", 1))
}

create_application_generator <- function(env, i) {
    name <- gsub(" ", "", paste("documents", i))

    env %>%
        add_generator(
            name_prefix = name,
            trajectory = create_trajectory(mu[i], name),
            distribution = function() rexp(1, lambda[i]),
            priority = i,
            preemptible = i + 1,
        )
}

add_generators <- function(env, n) {
    for (i in 1:n) {
        create_application_generator(env, i)
    }
}

In [13]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", preemptive = TRUE)

add_generators(env, P)

env %>% run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }

simmer environment: SuperDuperSim | now: 10000 | next: 10000.6691352204
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: documents1 | monitored: 1 | n_generated: 949 }
{ Source: documents2 | monitored: 1 | n_generated: 1007 }
{ Source: documents3 | monitored: 1 | n_generated: 1026 }
{ Source: documents4 | monitored: 1 | n_generated: 7068 }

In [14]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t2)) %>%
    release("server", 1)

In [15]:
arrivals <- env %>% get_mon_arrivals(ongoing = TRUE) # Show ongoing tasks too.
resources <- env %>% get_mon_resources()
arrivals
resources

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
documents40,1.773393,1.893377,0.1199846,TRUE,1
documents41,2.322054,2.970355,0.6483008,TRUE,1
documents42,4.437891,5.088018,0.6501270,TRUE,1
documents43,4.496584,6.139589,1.0515709,TRUE,1
documents10,7.862870,8.170349,0.3074790,TRUE,1
documents20,8.751629,8.897312,0.1456833,TRUE,1
documents44,9.454486,9.935456,0.4809696,TRUE,1
documents45,12.557795,13.566435,1.0086402,TRUE,1
documents46,12.936661,14.563889,0.9974542,TRUE,1


resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,1.773393,1,0,1,Inf,1,Inf,1
server,1.893377,0,0,1,Inf,0,Inf,1
server,2.322054,1,0,1,Inf,1,Inf,1
server,2.970355,0,0,1,Inf,0,Inf,1
server,4.437891,1,0,1,Inf,1,Inf,1
server,4.496584,1,1,1,Inf,2,Inf,1
server,5.088018,1,0,1,Inf,1,Inf,1
server,6.139589,0,0,1,Inf,0,Inf,1
server,7.862870,1,0,1,Inf,1,Inf,1


Отфильтруем NA значения:

In [16]:
arrivals["end_time"][arrivals["start_time"] == -1] <- SIMULATION_TIME
arrivals["start_time"][arrivals["start_time"] == -1] <- SIMULATION_TIME
arrivals["end_time"][is.na(arrivals["end_time"])] <- SIMULATION_TIME
arrivals["activity_time"][is.na(arrivals["activity_time"])] <- 0

Разделим заявки по приоритету и подсчитаем для каждого набора данных время:

In [21]:
patterns <- lapply(1:P, function(i) paste("documents", i, sep = ""))

results$practical <- lapply(patterns, function(pattern) {
    arrivals[grepl(arrivals$name, pattern = pattern, fixed = TRUE), ] %>%
        with(mean(end_time - activity_time - start_time))
})
patterns
results

[[1]]
[1] "documents1"

[[2]]
[1] "documents2"

[[3]]
[1] "documents3"

[[4]]
[1] "documents4"

,theoretical,practical
,<dbl>,<list>
1,5.786096,6.00358
2,4.225534,4.579203
3,3.370473,3.443136
4,1.104573,1.176621


### Вывод
Как видно, теоретически вычисленные значения с некоторой точностью совпадают
со значениями, полученными теоретически. При увеличении количества
экспериментов 𝑁 точность только увеличивается.